In [4]:
from pyspark.sql import Row
l=[Row(a=1,b=1),Row(a=1,b=1),Row(a=1,b=1)]
df=spark.createDataFrame(l)
df.show()

+---+---+
|  a|  b|
+---+---+
|  1|  1|
|  1|  1|
|  1|  1|
+---+---+



In [5]:
l=[Row(a=1,b=1),Row(a=1,b=1)]
df1=spark.createDataFrame(l)
df1.show()

+---+---+
|  a|  b|
+---+---+
|  1|  1|
|  1|  1|
+---+---+



In [7]:
df.join(df1,df.a == df1.a).show()

+---+---+---+---+
|  a|  b|  a|  b|
+---+---+---+---+
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
+---+---+---+---+



In [14]:
#from pyspark.sql import functions.join
df.join(df1,df.a == df1.a,'left_outer').show()

+---+---+---+---+
|  a|  b|  a|  b|
+---+---+---+---+
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
|  1|  1|  1|  1|
+---+---+---+---+



In [15]:
l=[Row(emp=1,name='venkat'),Row(emp=2,name='ramesh'),Row(emp=3,name='kumar')]
df_emp=spark.createDataFrame(l)
df_emp.show()

+---+------+
|emp|  name|
+---+------+
|  1|venkat|
|  2|ramesh|
|  3| kumar|
+---+------+



In [16]:
l=[Row(emp=1,sal=100),Row(emp=2,sal=200)]
df_sal=spark.createDataFrame(l)
df_sal.show()

+---+---+
|emp|sal|
+---+---+
|  1|100|
|  2|200|
+---+---+



In [24]:
final_df=df_emp.join(df_sal,df_emp.emp == df_sal.emp,'left_outer')
final_df.show()

+---+------+----+----+
|emp|  name| emp| sal|
+---+------+----+----+
|  1|venkat|   1| 100|
|  3| kumar|null|null|
|  2|ramesh|   2| 200|
+---+------+----+----+



In [29]:
final_df.select('*').show()  #All columns selected

+---+------+----+----+
|emp|  name| emp| sal|
+---+------+----+----+
|  1|venkat|   1| 100|
|  3| kumar|null|null|
|  2|ramesh|   2| 200|
+---+------+----+----+



In [30]:
#No columns selected
final_df.select().show()

++
||
++
||
||
||
++



In [45]:
#Important on defining alias , or ambigus column name
# used many function like alias , orderBy, join
# which can be used as lookup
# Both equi and non-equi join can be used

final_df=df_emp.alias('f_tb').join(df_sal.select('emp','sal').alias('s_tb'),df_emp.emp == df_sal.emp,'left_outer'). \
select('f_tb.emp','f_tb.name','s_tb.sal').orderBy('f_tb.emp')
final_df.show()

+---+------+----+
|emp|  name| sal|
+---+------+----+
|  1|venkat| 100|
|  2|ramesh| 200|
|  3| kumar|null|
+---+------+----+



In [64]:
#Lookup override option - filtering only the sal 200 in it.
final_df=df_emp.alias('f_tb').join(df_sal.select('emp','sal').filter("sal == '200'").alias('s_tb'),df_emp.emp == df_sal.emp,'left_outer'). \
select('f_tb.emp','f_tb.name','s_tb.sal').orderBy('f_tb.emp')
final_df.show()

+---+------+----+
|emp|  name| sal|
+---+------+----+
|  1|venkat|null|
|  2|ramesh| 200|
|  3| kumar|null|
+---+------+----+



In [65]:
#Router
# having more than filter would work

In [71]:
#Generating sequence number
from pyspark.sql.functions import monotonically_increasing_id
final_df.withColumn('T',monotonically_increasing_id()).show()
# But this is not a standred sequence number

+---+------+----+-----------+
|emp|  name| sal|          T|
+---+------+----+-----------+
|  1|venkat|null|          0|
|  2|ramesh| 200| 8589934592|
|  3| kumar|null|17179869184|
+---+------+----+-----------+



In [83]:
#Generating sequence number
from pyspark.sql.types  import *
from pyspark.sql.functions import lit
final_df=final_df.withColumn("T",lit(0)).show()

+---+------+----+---+
|emp|  name| sal|  T|
+---+------+----+---+
|  1|venkat|null|  0|
|  2|ramesh| 200|  0|
|  3| kumar|null|  0|
+---+------+----+---+



In [84]:
final_df.map(lambda x : final_df.T + 1 )

AttributeError: 'NoneType' object has no attribute 'map'

In [88]:
for i in range(len('hello')):
    print(i)

0
1
2
3
4


In [94]:
from pyspark.sql.Data  import foreach
def f(final_df):
    for i in final_df:
        print(final_df.name)
        
final_df.foreach(f)

ModuleNotFoundError: No module named 'pyspark.sql.DataFrame'

In [ ]:
# Iterate each rows in the DF